<a href="https://colab.research.google.com/github/alexisnpavlik/Job-listing-Query/blob/main/SONIC_Experiment_Alexis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set Up

In [1]:
# Prepare the environment
%%capture
from google.colab import drive, auth
drive.mount('/content/drive')
%cd /content/drive/MyDrive/python-library
!pip install .
!sudo apt-get install python-pymysql
!pip install colab-env
!pip install Hashids
import colab_env
%cd /content
import nltk
nltk.download('stopwords')
import gallifrey
import pandas as pd
import numpy as np
import time
from hashids import Hashids

#Set up for Hash calculation
hasher = Hashids("jS5qAlR0kHbAeqeuVowPBnyotKuTvvlGzIzkEtz79XQGB7BJHb", 
                 8, 
                 "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890")

# Authenticate
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials as GC
gc = gspread.authorize(GC.get_application_default())

# create, and save df
from gspread_dataframe import set_with_dataframe

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=qSoJLfrBG1wlIL8EfM4hI6nC8v09NG&prompt=consent&access_type=offline&code_challenge=CzeqOqDKhkt1__KVJjYF7QWN7RZ08GRIJARv2vmgLhI&code_challenge_method=S256

Enter verification code: 4/1AX4XfWh6XgKmQiq-ibpCDTfLVrRu_AA7NMQB4B0IV4tPPPwZMdq4UtkX6cs


## Manual Update

#### Query


In [63]:
!rm -r Job-listing-Query
!git clone https://github.com/alexisnpavlik/Job-listing-Query.git

main_query=open("""/content/Job-listing-Query/main_query.sql""", 'r').read()
organizations_query=open("""/content/Job-listing-Query/organizations_query.sql""", 'r').read()

Cloning into 'Job-listing-Query'...
remote: Enumerating objects: 265, done.
remote: Counting objects: 100% (265/265), done.
remote: Compressing objects: 100% (184/184), done.
remote: Total 265 (delta 160), reused 180 (delta 79), pack-reused 0
Receiving objects: 100% (265/265), 38.73 KiB | 4.30 MiB/s, done.
Resolving deltas: 100% (160/160), done.


In [64]:
start_time=time.time()
main = pd.DataFrame(gallifrey.discovery.execute_query(main_query))
organizations_df = pd.DataFrame(gallifrey.solar.execute_query(organizations_query))
print('Tiempo de ejecución:',time.time()-start_time,'s')

Tiempo de ejecución: 211.76552295684814 s


In [65]:
main_df = main.copy(deep=True)
main_df

,ID,Job title,Company_id,location,Created date,Approved date,Commited date,Applicant Acquisition Coordinator,Type of job,Type of service,Language of the post,Status,Last changes,Closing Date,Mutual Matches,Disqualified,completed applications,Incomplete applications,Others,Active,Pending for review,Hires,Sharing token
0,1854259,Líder técnico (México),332263.0,Mexico,2022-02-01,2022-02-01,2022-02-01,Juan David Pinilla Boude,full-time,prime,en,open,2022-02-01,2022-03-03,0.0,9.0,0.0,9.0,0.0,0.0,0.0,None,8m9uyYRgfLzK
1,1844230,Arquitecto de Software,468884.0,None,2022-01-31,2022-02-01,2022-01-31,Viviana Trujillo Estrada,full-time,prime,en,open,2022-02-01,2022-03-02,22.0,110.0,22.0,110.0,0.0,22.0,0.0,None,Byq97rdFvc6A
2,1844224,Diseñador UI/UX,730852.0,None,2022-01-31,2022-01-31,2022-02-01,Juan Felipe Bonilla García,full-time,self_service,en,open,2022-02-01,2022-03-02,21.0,63.0,21.0,63.0,0.0,21.0,0.0,None,ecRBwFZBD0f6
3,1844222,Website Builder,748394.0,Argentina,2022-01-31,2022-01-31,2022-02-01,Melisa Medina Gomez,full-time,prime,es,open,2022-02-01,2022-03-02,12.0,40.0,12.0,40.0,12.0,24.0,-12.0,None,H0l5WuzmplNA
4,1844221,Quality Support,748394.0,Argentina,2022-01-31,2022-01-31,2022-02-01,Melisa Medina Gomez,full-time,prime,es,open,2022-02-01,2022-03-02,60.0,108.0,60.0,108.0,0.0,60.0,0.0,None,bzCg2dkmMqC6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3906,44516,Travel Industry Blogger / Writer,592682.0,None,2020-06-15,2020-06-15,2021-08-13,Camilo Alvarez Motta,part-time,self_service,en,closed,2021-10-09,2021-08-23,1026.0,9666.0,9558.0,9666.0,459.0,1485.0,8073.0,2021-08-17,Bh2XHUXmeVAx
3907,44274,Comercial Independiente en Seguros de vida - T...,691035.0,Colombia,2020-06-13,2020-06-13,2021-08-03,Jenny Patricia Castro,part-time,self_service,es,closed,2022-01-25,2022-01-25,21006.0,29376.0,27216.0,29376.0,1458.0,22464.0,4752.0,2021-06-28,RFP8bUKMD1gD
3908,39762,Front-end Developer,739149.0,None,2020-05-22,2020-05-22,2021-09-09,David Camargo,full-time,self_service,en,closed,2021-10-05,2020-06-22,84.0,126.0,112.0,126.0,0.0,84.0,28.0,None,Z9eRj6gJpRVC
3909,8683,Full-stack Developer,739149.0,None,2019-12-20,2019-12-20,2021-09-09,David Camargo,full-time,self_service,en,closed,2021-10-05,2021-01-05,187.0,385.0,209.0,385.0,88.0,275.0,-66.0,None,VRbyNQ4TJJ16


In [66]:
organizations_df.head(10)

,Company_id,Company
0,795908,
1,773085,0 2 Convergencia Digital S A S
2,763230,2 Way Sports
3,795148,23Ky Colombia S.A.S.
4,891553,60 Decibels
5,760653,Aberdare College
6,761403,"Absolute Computer Solutions, Inc."
7,805948,ABUSA Lab
8,786038,Academia de Inglés de la Universidad de Tarapacá
9,786905,Academia de Música DEUS


In [67]:
# Alfa ID Generator
def alfa_id(id):
  return hasher.encode(id)

# URL Generator
def url_generator(id):
  return 'https://torre.co/postings/'+hasher.encode(id)

# Manager URL Generator
def manager_generator(id):
  return 'https://torre.co/postings/'+hasher.encode(id)+'/members-list?sharingToken='

# Transforms columns
main_df.insert(1,"Alfa ID", main_df['ID'].apply(alfa_id),True)
main_df.insert( 2,"URL", main_df['ID'].apply(url_generator),True)

main_df.insert( 6,"Manager link", 'https://torre.co/postings/'+ main_df['Alfa ID'] +'/members-list?sharingToken='+ main_df['Sharing token'] ,True)

In [68]:
opportunities_df = pd.merge(left=main_df,right=organizations_df)

opportunities_df = opportunities_df.reindex(columns=['ID','Job title','Company','Alfa ID','location','Created date','Approved date','Commited date','Applicant Acquisition Coordinator','Type of job',
                                             'Type of service','URL','Manager link','Language of the post','Status','Mutual Matches','Disqualified','Last changes','Closing Date',
                                             'Completed Applications','Incomplete applications','Others','Active','Pending for review','Hires'])

In [69]:
sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1V1RkMTrOFEQLsYGJaA9KukO9fgtgf6uwG8o-QHrkk2Q/edit#gid=0').sheet1

sheet.clear()

set_with_dataframe(sheet, opportunities_df) 

# Update Google Sheet

In [ ]:
sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1V1RkMTrOFEQLsYGJaA9KukO9fgtgf6uwG8o-QHrkk2Q/edit#gid=0').sheet1

sheet.clear()

set_with_dataframe(sheet, opportunities_df) 

# Save to Google drive

##### Save csv

In [ ]:
drive.mount('/content/drive')

In [ ]:
opportunities_df.to_csv('/content/drive/My Drive/Data_studio_dataset/Dataset.csv',index=False,encoding='utf-8')

##### Save  Excel

In [ ]:
drive.mount('/content/drive')

In [ ]:
opportunities_df.to_excel('/drive/My Drive/Data_studio_dataset/Dataset.xls',index=False,encoding='utf-8')

# Testing

In [ ]:
query_testing = """

select id as 'Company_id', name as 'Company' from organizations ;

"""

In [ ]:
Testing = pd.DataFrame(gallifrey.solar.execute_query(query_testing))

### Update Google Sheet

In [ ]:
sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1V1RkMTrOFEQLsYGJaA9KukO9fgtgf6uwG8o-QHrkk2Q/edit#gid=0').sheet1

sheet.clear()

set_with_dataframe(sheet, Testing) 

In [ ]:
Testing.head()

,Company_id,Company
0,795908,
1,773085,0 2 Convergencia Digital S A S
2,763230,2 Way Sports
3,795148,23Ky Colombia S.A.S.
4,891553,60 Decibels
